In [1]:
# Coursera Capston - Applied Data Science

## Gary S. Netherton - June 30, 2020

### This notebook will be used for the capstone project for the Applied Data Science coursework.

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
